In [1]:
# ==============================================================================
# 1. SETUP E IMPORTAÇÕES
# ==============================================================================
import sys
import glob
import json
import os
import fitz  # PyMuPDF
from tqdm import tqdm # Para barras de progresso

# Adiciona o diretório raiz ao path
if '..' not in sys.path:
    sys.path.append('..')

# Importa todas as nossas funções especializadas de 'src'
from src.image_extractor import extract_images_from_pdf
from src.table_extractor import extract_raw_dataframe
from src.table_enhancer import enhance_table             # Parser "Genérico" (ou de Calendário)
from src.horario_parser import extract_schedule_from_page # Parser de Horários
from src.ppc_parser import parse_ppc_page                 # >>> NOVO: Parser de PPC

from src.text_normalization import normalize_text
from src.structure_detector import detect_structure
from src.deduplicator import deduplicate_chunks
from src.metadata_enricher import enrich_with_metadata

print("Módulos e funções importados com sucesso.")

Módulos e funções importados com sucesso.


In [2]:
# ==============================================================================
# 2. DEFINIÇÃO DE CAMINHOS E CONFIGURAÇÕES
# ==============================================================================
# Caminhos de entrada e saída
input_dir = '../data/input'
output_dir = '../data/output'
images_output_dir = os.path.join(output_dir, 'images') # Subpasta para imagens

# Cria os diretórios de saída se eles não existirem
os.makedirs(output_dir, exist_ok=True)
os.makedirs(images_output_dir, exist_ok=True)

# Carrega o dicionário de siglas de um arquivo de configuração externo
acronyms_path = '../data/acronyms.json'
try:
    # Corrigido: Removido espaço estranho antes de with
    with open(acronyms_path, 'r', encoding='utf-8') as f:
        dicionario_de_siglas = json.load(f)
    print(f"Dicionário de siglas carregado de '{acronyms_path}'.")
except FileNotFoundError:
    print(f"Aviso: Arquivo de siglas '{acronyms_path}' não encontrado. A expansão não será realizada.")
    dicionario_de_siglas = {}
except json.JSONDecodeError as e: # Adicionado: Captura erro de JSON mal formatado
    print(f"Erro ao ler o arquivo JSON de siglas '{acronyms_path}': {e}. A expansão não será realizada.")
    dicionario_de_siglas = {}

Dicionário de siglas carregado de '../data/acronyms.json'.


In [3]:
# ==============================================================================
# 3. EXECUÇÃO DO PIPELINE COMPLETO (VERSÃO FINAL COM DETECÇÃO AVANÇADA DE TIPO)
# ==============================================================================
import pandas as pd
import fitz
import os
import json

# Encontra todos os arquivos PDF na pasta de entrada
pdf_files = glob.glob(os.path.join(input_dir, '*.pdf'))
print(f"\nIniciando o processamento de {len(pdf_files)} arquivo(s) PDF...")

# Itera sobre cada arquivo PDF encontrado
for pdf_path in tqdm(pdf_files, desc="Processando Documentos"):
    file_name = os.path.basename(pdf_path)
    print(f"\nProcessando: {file_name}")

    # --- >>> INÍCIO: Detecção Automática de Tipo de PDF por Conteúdo <<< ---
    pdf_type = "generic" # Começa com o padrão
    try:
        with fitz.open(pdf_path) as temp_doc:
            if len(temp_doc) > 0:
                # Analisa o texto da primeira página em minúsculas
                first_page_text = temp_doc.load_page(0).get_text("text").lower()

                # 1. Lista de keywords para PPC (Prioridade 1)
                ppc_keywords = ["projeto pedagógico", "matriz curricular", "ementário", "colegiado de curso", "ppcbcc","Projeto Pedagógico"]
                # 2. Lista de keywords para Horário (Prioridade 2)
                horario_keywords = ["horário", "segunda", "terça", "quarta", "quinta", "sexta", "manhã", "tarde"]
                
                # --- Lógica de Prioridade ---
                # 1. Checa se é PPC (keywords mais fortes)
                ppc_match_count = sum(1 for kw in ppc_keywords if kw in first_page_text)
                if ppc_match_count >= 1: # Se 2 ou mais keywords de PPC baterem
                    pdf_type = "ppc"
                    print(f"--> Detectado como tipo 'ppc' (Projeto Pedagógico). Keywords: {[kw for kw in ppc_keywords if kw in first_page_text]}")
                
                # 2. Se NÃO for PPC, checa se é Horário
                elif sum(1 for kw in horario_keywords if kw in first_page_text) >= 5: # Usando seu threshold de 5
                    pdf_type = "schedule"
                    print(f"--> Detectado como tipo 'schedule' (horário).")
                
                else:
                    # Se não for nenhum dos dois, continua como "generic"
                    print(f"--> Detectado como tipo 'generic' (lógica antiga/padrão será usada).")
            else:
                 print("--> PDF vazio, tratando como 'generic'.")
    except Exception as e:
        print(f"Alerta: Não foi possível analisar conteúdo de '{file_name}': {e}. Tratando como 'generic'.")
    # --- >>> FIM: Detecção Automática de Tipo de PDF <<< ---


    # --- ETAPA DE EXTRAÇÃO (NÍVEL DO DOCUMENTO) ---
    all_raw_text = ""
    page_level_data = [] # Lista temporária para dados de página
    metadata = {         # Metadados base (serão atualizados depois)
        "doc_id": file_name.replace('.pdf', ''),
        "nome_doc": file_name,
    }

    try:
        # Abre o PDF principal para processamento completo
        with fitz.open(pdf_path) as doc:
            num_pages = doc.page_count
            for page_num in range(num_pages):
                page = doc.load_page(page_num)
                page_1_indexed = page_num + 1

                # Extração de texto bruto (para content_chunks)
                page_text = page.get_text("text")
                if page_text:
                     all_raw_text += page_text + "\n\n"

                # Extração de Imagens (comum a todos)
                image_info = extract_images_from_pdf(pdf_path, page_1_indexed, images_output_dir)

                # --- >>> INÍCIO: LÓGICA CONDICIONAL DE PROCESSAMENTO <<< ---
                
                if pdf_type == "schedule":
                    # --- LÓGICA PARA HORÁRIO (Chama horario_parser.py) ---
                    schedule_page_data = extract_schedule_from_page(pdf_path, page_1_indexed)
                    page_data_entry = {"page": page_1_indexed, "page_type": "schedule", "images": image_info}
                    if schedule_page_data:
                        schedule_page_data.pop("pagina", None)
                        page_data_entry.update(schedule_page_data)
                    else:
                        page_data_entry["error"] = "Falha na extração/processamento do horário nesta página."
                    page_level_data.append(page_data_entry)

                elif pdf_type == "ppc":
                    # --- LÓGICA PARA PPC (Chama ppc_parser.py) ---
                    # (Usando nosso placeholder por enquanto)
                    ppc_page_data = parse_ppc_page(pdf_path, page_1_indexed)
                    page_data_entry = {"page": page_1_indexed, "images": image_info}
                    page_data_entry.update(ppc_page_data) # Adiciona 'page_type', 'tables', etc. do parser
                    page_level_data.append(page_data_entry)

                else: # pdf_type == "generic"
                    # --- LÓGICA GENÉRICA (ANTIGA / table_enhancer.py) ---
                    raw_df = extract_raw_dataframe(pdf_path, page=page_1_indexed)
                    enhanced_table_info = {}
                    if not raw_df.empty:
                        try:
                            enhanced_table_info = enhance_table(raw_df)
                        except Exception as table_enhance_error:
                             print(f"\nAlerta: Erro ao aprimorar tabela genérica na pág {page_1_indexed}: {table_enhance_error}")
                             enhanced_table_info = {"error": str(table_enhance_error)}
                    
                    page_level_data.append({
                        "page": page_1_indexed,
                        "page_type": "generic",
                        "tables": [enhanced_table_info.get("cleaned_table", [])] if "error" not in enhanced_table_info else [{"error": enhanced_table_info.get("error")}],
                        "table_legends": [enhanced_table_info.get("legend", "")] if "error" not in enhanced_table_info else [],
                        "table_summaries": [enhanced_table_info.get("summary", "")] if "error" not in enhanced_table_info else [],
                        "images": image_info
                    })
                # --- >>> FIM: LÓGICA CONDICIONAL DE PROCESSAMENTO <<< ---

    except Exception as e:
        print(f"\nERRO CRÍTICO ao processar páginas de '{file_name}'. Pulando para próximo arquivo. Detalhes: {e}")
        continue # Pula para o próximo arquivo PDF

    # --- ETAPAS DE PROCESSAMENTO (TEXTO COMPLETO DO DOCUMENTO) ---
    final_chunks = []
    if all_raw_text:
        try:
            normalized_text = normalize_text(all_raw_text, acronyms=dicionario_de_siglas)
            structured_chunks = detect_structure(normalized_text) # (Ainda não otimizado para PPC)
            unique_chunks = deduplicate_chunks(structured_chunks)
        except Exception as text_processing_error:
             print(f"Alerta: Erro no processamento de texto completo para '{file_name}': {text_processing_error}")
             unique_chunks = [{"error": "Falha no processamento do texto completo", "details": str(text_processing_error)}]
    else:
        print(f"Alerta: Nenhum texto bruto extraído de '{file_name}'.")
        unique_chunks = [{"error": "Nenhum texto bruto extraído do PDF"}]


    # --- AJUSTE PARA MOVER HORÁRIOS PARA METADATA ---
    schedules_found = []
    final_page_specific_data = []

    for page_data in page_level_data:
        if page_data.get("page_type") == "schedule" and "horario" in page_data and page_data.get("horario") is not None and "error" not in page_data:
            schedule_info = {
                "pagina_origem": page_data.get("page"),
                "semestre": page_data.get("semestre"),
                "turma": page_data.get("turma"),
                "salas_info": page_data.get("salas_info"),
                "horario": page_data.get("horario")
            }
            schedules_found.append(schedule_info)
            final_page_specific_data.append({
                "page": page_data.get("page"),
                "page_type": "schedule_processed",
                "images": page_data.get("images", [])
            })
        else:
            final_page_specific_data.append(page_data) # Mantém PPC, Genérico, ou erros de horário

    if schedules_found:
      metadata["schedules"] = schedules_found
    # --- FIM AJUSTE METADATA ---

    # Enriquece os chunks de texto DEPOIS de finalizar metadata
    if unique_chunks and isinstance(unique_chunks[0], dict) and "error" not in unique_chunks[0]:
         try:
             final_chunks = enrich_with_metadata(unique_chunks, metadata)
         except Exception as enrich_error:
              print(f"Alerta: Erro ao enriquecer chunks de texto: {enrich_error}")
              final_chunks = [{"error": "Falha ao enriquecer chunks de texto", "details": str(enrich_error)}]
    elif not unique_chunks:
         final_chunks = []
    else: # Caso unique_chunks já contenha um erro
         final_chunks = unique_chunks 


    # --- MONTAGEM DO JSONL FINAL ---
    documento_final = {
        "metadata": metadata,
        "page_specific_data": final_page_specific_data,
        "content_chunks": final_chunks
    }

    # --- SAÍDA ---
    output_path_jsonl = os.path.join(output_dir, f"{file_name}.jsonl")
    try:
        print(f"---> PREPARANDO PARA SALVAR {file_name} (Tipo: {pdf_type}):")
        print(f"     Metadata keys: {list(documento_final.get('metadata', {}).keys())}")
        print(f"     page_specific_data length: {len(documento_final.get('page_specific_data', []))}")
        print(f"     content_chunks length: {len(documento_final.get('content_chunks', []))}")

        with open(output_path_jsonl, 'w', encoding='utf-8') as f:
            f.write(json.dumps(documento_final, ensure_ascii=False, indent=2)) # Salva formatado
        print(f"---> Arquivo JSONL salvo com sucesso (formatado com indentação): {output_path_jsonl}")
    except Exception as e:
        print(f"\nERRO ao salvar o arquivo de saída '{output_path_jsonl}'. Detalhes: {e}")

# --- Fim do loop principal ---

print("\n----------------------------------------------------")
print("Processamento de todos os arquivos concluído!")
print(f"Resultados salvos em: '{output_dir}'")


Iniciando o processamento de 2 arquivo(s) PDF...


Processando Documentos:   0%|                                                                    | 0/2 [00:00<?, ?it/s]


Processando: Ciencia-da-computacao-01-2025_com_sala_MkIII.pdf
--> Detectado como tipo 'schedule' (horário).

--- Processando Página 1 ---
Metadados encontrados: {'semestre': '1/2025', 'turma': '20 INGRESSANTES (MATUTINO)', 'salas_info': 'P2 – Sala 7, P2-LAB CC (quando indicado)'}
----> Sala padrão detectada: 'P2 – Sala 7'
Tabela da página 1 processada com sucesso (salas vazias preenchidas).

--- Processando Página 2 ---
Metadados encontrados: {'semestre': '1/2025', 'turma': '30 (MATUTINO)', 'salas_info': 'P2 - Sala 9, P2-LAB CC (quando indicado)'}
----> Sala padrão detectada: 'P2 - Sala 9'
Tabela da página 2 processada com sucesso (salas vazias preenchidas).

--- Processando Página 3 ---
Metadados encontrados: {'semestre': '1/2025', 'turma': '50 PERÍODO (VESPERTINO)', 'salas_info': 'P2 – sala 9, P2-LAB CC (quando indicado)'}
----> Sala padrão detectada: 'P2 – sala 9'
Tabela da página 3 processada com sucesso (salas vazias preenchidas).

--- Processando Página 4 ---
Metadados encontrad

Processando Documentos:  50%|██████████████████████████████                              | 1/2 [00:03<00:03,  3.45s/it]

----> Sala padrão detectada: 'P2 – sala 7'
Tabela da página 7 processada com sucesso (salas vazias preenchidas).
---> PREPARANDO PARA SALVAR Ciencia-da-computacao-01-2025_com_sala_MkIII.pdf (Tipo: schedule):
     Metadata keys: ['doc_id', 'nome_doc', 'schedules']
     page_specific_data length: 7
     content_chunks length: 1
---> Arquivo JSONL salvo com sucesso (formatado com indentação): ../data/output\Ciencia-da-computacao-01-2025_com_sala_MkIII.pdf.jsonl

Processando: PPCBCC2019.pdf
--> Detectado como tipo 'ppc' (Projeto Pedagógico). Keywords: ['projeto pedagógico']


C:\Users\extre\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (546.36, 549.36) does not lie in column range (50.14801655078598, 546.009939540508)
  warnings.warn(
C:\Users\extre\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (546.36, 549.36) does not lie in column range (50.149422517382106, 546.009939540508)
  warnings.warn(
C:\Users\extre\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (546.36, 549.36) does not lie in column range (50.14895386185007, 546.009939540508)
  warnings.warn(
C:\Users\extre\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (546.36, 549.36) does not lie in column range (50.149422517382106, 546.009939540508)
  warnings.warn(
C:\Users\extre\AppData\Local\Programs\Python\Python313\Lib\site-packages\camelot\utils.py:1217: UserWarning:   (546.36, 549.36) does not lie in column

Info PPC_PARSER: _parse_optativas não encontrou 'DISCIPLINA'. Tratando como página de continuação.


Processando Documentos: 100%|████████████████████████████████████████████████████████████| 2/2 [01:10<00:00, 35.29s/it]

---> PREPARANDO PARA SALVAR PPCBCC2019.pdf (Tipo: ppc):
     Metadata keys: ['doc_id', 'nome_doc']
     page_specific_data length: 103
     content_chunks length: 1
---> Arquivo JSONL salvo com sucesso (formatado com indentação): ../data/output\PPCBCC2019.pdf.jsonl

----------------------------------------------------
Processamento de todos os arquivos concluído!
Resultados salvos em: '../data/output'
